In [1]:
import pandas as pd
import numpy as np
import torch
import pickle
import tqdm
import random
import nlp_utils as nu
from collections import Counter
from torchtext import datasets
from torch.utils import data

pickle_path = "../Pickles/"
our_training_path = "../Data/train2.csv"
our_testing_path = "../Data/val2.csv"
save_path = pickle_path

our_training_csv = pd.read_csv(our_training_path)
our_testing_csv = pd.read_csv(our_testing_path)

In [2]:
with open(pickle_path+"idx2utt.pickle","rb") as f:
    idx2utt = pickle.load(f)
with open(pickle_path+"utt2idx.pickle","rb") as f:
    utt2idx = pickle.load(f)
    
with open(pickle_path+"idx2emo.pickle","rb") as f:
    idx2emo = pickle.load(f)
with open(pickle_path+"emo2idx.pickle","rb") as f:
    emo2idx = pickle.load(f)
    
with open(pickle_path+"idx2speaker.pickle","rb") as f:
    idx2speaker = pickle.load(f)
with open(pickle_path+"speaker2idx.pickle","rb") as f:
    speaker2idx = pickle.load(f)

In [15]:
batch_size = 8
seq_len = 5
seq2_len = seq_len
emb_size = 768
hidden_size = 768
batch_first = True

##################
### Reading training data
X_train = []
X_train_d_id = []
X_train_emo = []
y_train_flip = []

X_train_tmp = []
X_train_d_id_tmp = []
X_train_emo_tmp = []
y_train_flip_tmp = []

global_speaker_info = {}
speaker_dialogues = {}
speaker_emotions = {}
speaker_indices = {}
utt_len = {}

d_id = 0
c_id = 0
prev_d_id=0

In [16]:
for i in range(len(our_training_csv)):
    if our_training_csv["Dialogue_Id"][i] != prev_d_id:
        if len(X_train_tmp) > seq_len:
            utt_len[d_id] = seq_len
        else:
            utt_len[d_id] = len(X_train_tmp)
          
        if len(X_train_tmp) < seq_len:
            for k in range(len(X_train_tmp),seq_len):
                X_train_tmp.append(utt2idx["<pad>"])
                X_train_d_id_tmp.append(d_id)
                X_train_emo_tmp.append(emo2idx["neutral"])
                y_train_flip_tmp.append(0)
        else:
            diff = len(X_train_tmp)-seq_len
            X_train_tmp = X_train_tmp[diff:]
            X_train_d_id_tmp = X_train_d_id_tmp[diff:]
            X_train_emo_tmp = X_train_emo_tmp[diff:]
            y_train_flip_tmp = y_train_flip_tmp[diff:]
            
        if len(global_speaker_info[d_id].keys()) < seq_len:
            for k in range(len(global_speaker_info[d_id].keys()),seq_len):
                global_speaker_info[d_id][k] = speaker2idx["<pad>"]
                speaker_emotions[d_id][k] = emo2idx["neutral"]
        else:
            tmp_speaker_info = global_speaker_info[d_id].copy()
            tmp_speaker_emotions = speaker_emotions[d_id].copy()
            
            for k_i,k in enumerate(range(diff,len(global_speaker_info[d_id].keys()))):
                global_speaker_info[d_id][k_i] = tmp_speaker_info[k]
                speaker_emotions[d_id][k_i] = tmp_speaker_emotions[k]
        
        for every_sp in speaker_dialogues[d_id].keys():
            if len(speaker_dialogues[d_id][every_sp]) < seq_len:
                for k in range(len(speaker_dialogues[d_id][every_sp]),seq_len):
                    speaker_dialogues[d_id][every_sp].append(utt2idx["<pad>"])

        X_train.append(X_train_tmp)
        X_train_d_id.append(X_train_d_id_tmp)
        X_train_emo.append(X_train_emo_tmp)
        y_train_flip.append(y_train_flip_tmp)
        
        X_train_tmp = []
        X_train_d_id_tmp = []
        X_train_emo_tmp = []
        y_train_flip_tmp = []

        d_id += 1
        c_id = 0
        prev_d_id = prev_d_id + 1
    if our_training_csv["Dialogue_Id"][i] == prev_d_id:
        if d_id not in global_speaker_info.keys():
            global_speaker_info[d_id] = {}
            speaker_dialogues[d_id] = {}
            speaker_emotions[d_id] = {}
            speaker_indices[d_id] = {}

        utt = utt2idx[nu.preprocess_text(our_training_csv["Utterance"][i])]
        sp = speaker2idx[our_training_csv["Speaker"][i]]
        flip = float(our_training_csv["Annotate(0/1)"][i])
        emo = emo2idx[our_training_csv["Emotion_name"][i]]
        if np.isnan(flip):
            flip = 0
        
        X_train_tmp.append(utt)
        X_train_d_id_tmp.append(d_id)
        X_train_emo_tmp.append(emo)
        y_train_flip_tmp.append(flip)
        
        global_speaker_info[d_id][c_id] = sp
        if sp in speaker_dialogues[d_id].keys():
            speaker_dialogues[d_id][sp].append(utt)
        else:
            speaker_dialogues[d_id][sp] = [utt]
        speaker_emotions[d_id][c_id] = emo
        if sp in speaker_indices[d_id].keys():
            speaker_indices[d_id][sp].append(c_id)
        else:
            speaker_indices[d_id][sp] = [c_id]
        
        c_id += 1

In [17]:
D = torch.LongTensor(X_train_d_id)
X = torch.LongTensor(X_train)
E = torch.LongTensor(X_train_emo)
Y = torch.LongTensor(y_train_flip)

In [23]:
print(X_train)

[[6324, 3706, 5304, 757, 3089], [6714, 1723, 7725, 7979, 4418], [7979, 4418, 3100, 765, 1097], [3100, 765, 1097, 1729, 6855], [7495, 3920, 3736, 9125, 9125], [3736, 5009, 5521, 324, 7708], [9341, 2478, 4507, 8700, 6004], [2478, 4507, 8700, 6004, 6251], [8700, 6004, 6251, 1167, 936], [6004, 6251, 1167, 936, 7285], [2139, 2499, 1324, 9125, 9125], [2499, 1324, 404, 5720, 4134], [1324, 404, 5720, 4134, 7609], [404, 5720, 4134, 7609, 5596], [841, 3173, 521, 388, 5181], [3173, 521, 388, 5181, 4914], [738, 7855, 2473, 1875, 258], [7436, 6424, 7904, 10170, 6227], [10170, 6227, 4428, 5403, 2574], [8550, 6316, 1314, 4314, 9125], [4314, 9336, 1255, 8143, 4447], [9336, 1255, 8143, 4447, 748], [8143, 4447, 748, 8905, 5289], [8905, 5289, 5435, 2962, 3304], [2470, 207, 23, 1676, 5789], [1676, 5789, 9723, 5495, 1382], [5789, 9723, 5495, 1382, 8547], [1382, 8547, 1045, 3484, 9383], [8547, 1045, 3484, 9383, 3303], [9383, 3303, 9925, 9508, 9451], [9508, 9451, 2961, 4721, 4161], [4721, 4161, 2471, 10054, 